In [2]:
import tensorflow as tf
import numpy as np

# Ideas:
'''
Agent must correlate between the sequence of questions, themes and response given so far
in relation with the cluster scores
'''

In [20]:
# These are maximum ids in the dataset, used for embedding models (vocabulary size !)
MAX_NUM_QUESTIONS = 200
MAX_NUM_THEMES = 20

# Size of the embeddings
QUESTION_ID_EMBSIZE = 8
THEME_ID_EMBSIZE = 4
NUM_CLUSTERS = 4 #-- Maybe try to keep as dynamic variable 



setOfAvailableQuestions = np.random.choice(MAX_NUM_QUESTIONS, size=(100))
setOfAvailableThemes = np.random.choice(MAX_NUM_THEMES, size=(5))

class MyEmbModel(tf.keras.Model):
    def __init__(self, setOfAvailableQuestions, setOfAvailableThemes):
        super(MyEmbModel, self).__init__()
        self.embQuestionId = tf.keras.layers.Embedding(MAX_NUM_QUESTIONS, QUESTION_ID_EMBSIZE)
        self.emdThemeId = tf.keras.layers.Embedding(MAX_NUM_THEMES, THEME_ID_EMBSIZE)
        self.questionFC = tf.keras.layers.Dense(32)
        
        self.setOfAvailableQuestions = setOfAvailableQuestions
        self.setOfAvailableThemes = setOfAvailableThemes
        
        self.head_themes = tf.keras.layers.Dense(len(self.setOfAvailableQuestions))
        self.head_questions = tf.keras.layers.Dense(len(self.setOfAvailableThemes))
        
    # input_dat is a tuple of :
    #   A. sequence of themes and questions + their devscore so far
    # input_data[0] = [None, [(theme_0, question_0, devscore), (theme_0, question_1),... .]]
    #  (theme_1, question_0), ....]
    #  B. Scores (probabilities) of agent for each cluster type,  at each time step:
    #  input_data[1] = [None, [scores_time0:[prob cluster_0, cluster_1], scores_time1:[...]]]
    #  Basically the len of A is equal to B
    
    
    # GRU OF 
    
    # [ timestep_i :  [themeIdEmb, questionIdEmb, devScore for this question, probs after each question] => 32 neurons].
    # Input of 32 units, output of 32 units + [Head1: Softmax(FC(numThemes)) Head2: Softmax(FC(numQuestions))]
    # selectTheme should be true if the mobel is required to select a theme, or false if should select a question inside the theme
    def call(self, input_data, selectTheme : bool):
        sequenceOfQuestions = input_data[0]
        clusterScoresAfterQuestion = input_data[1]
        # Same timesteps size ?
        assert len(sequenceOfQuestions) == len(clustersScoresAfterQuestions)
        # Same timesteps per question
        #assert len(sequenceOfQuestions[0] == len(clustersScoresAfterQuestions[0]))
        
        
        # TODO: this is for a single time step but we should do this inside a GRU
        timestep_questionData = input_data[0][0]
        timestep_scoresAfterQuestion = input_data[0][0]
        
        themeId, questionId, devScore = firstTimestep
        themeIdEmb = self.embThemeId(tf.reshape(themeId))
        questionIdEmb = self.embQuestionId(questionId)
        
        questionAndResp_interm = tf.concat([themeIdEmb, questionIdEmb, devScore, timestep_scoresAfterQuestion], axis=-1)
        questionAndResp_res = self.questionFC(questionAndResp_interm)
        
        # After all GRU steps...
        res = None
        if selectTheme:
            res = self.head_themes()
        else:
            res = self.head_questions()
        res = tf.nn.softmax(res)
        
        return res
    


In [21]:
# Creates a random dataset for test
def randomDataset(num_timesteps, num_clusters):
    res = {}
    
    # Get some random questions and theme id 
    setOfAvailableQuestions = np.random.choice(MAX_NUM_QUESTIONS, size=(100))
    setOfAvailableThemes = np.random.choice(MAX_NUM_THEMES, size=(5))
    
    res['setOfAvailableQuestions'] = setOfAvailableQuestions
    res['setOfAvailableThemes'] = setOfAvailableThemes
    
    # Randomize some themes and question ids
    themes = np.random.randint(low=0, high=MAX_NUM_THEMES, size=(num_timesteps))
    questions = np.random.randint(low=0, high=MAX_NUM_QUESTIONS, size=(num_timesteps))
    devscores = np.random.uniform(low=-7, high=7, size=[num_timesteps])
    
    questions_data = list(zip(themes, questions, devscores))
    clusterprobs_data = np.random.uniform(low=0, high=1, size=[num_timesteps, num_clusters])
    timestemValues = (questions_data, clusterprobs_data)

    res['timestemValues'] = timestemValues
    
    return res
    
    
datasetSample = randomDataset(num_timesteps=10, num_clusters=NUM_CLUSTERS)

embModel = MyEmbModel(setOfAvailableQuestions=datasetSample['setOfAvailableQuestions'],
                     setOfAvailableThemes=datasetSample['setOfAvailableThemes'])



def selectTheme(model, data):
    model(input_data=data['timestemValues'], selectTheme=True)

def selectQuestion(model, data):
    model(input_data=data['timestemValues'], selectTheme=True)


nextTheme = selectTheme(embModel, datasetSample)
nextQuestion = selectQuestion(embModel, datasetSample)


NameError: name 'clustersScoresAfterQuestions' is not defined